In [ ]:
%%capture
!pip install transformers datasets

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from torch.utils.data import DataLoader
from tabulate import tabulate
from datasets import load_dataset

from tqdm.notebook import tqdm
from transformers import BertTokenizer

This is a template of the notebook that you should complete and enrich with your own code.

First cells will be the same than the ones of the lab on text convolution.

# Data loading


In [ ]:
dataset = load_dataset("scikit-learn/imdb", split="train")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

IMDB Dataset.csv:   0%|          | 0.00/66.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})


# Pre-processing / Tokenization

This is a very important step. It maybe boring but very important. In this session we will be lazy, but in real life, the time spent on inspecting and cleaning data is never wasted. It is true for text, but also for everything.



In PyTorch, everything is tensor. Words are replaced by indices. A sentence, is therefore a sequence of indices (long integers). In the first HW, you constructed a `WhiteSpaceTokenizer`. Here we will use an already built tokenizer. It is more appropriate to transformers. It relies on sub-word units, and converts everything in lower case. This is not always the best choice, but here it will be sufficient. To quote the documentation, this tokenizer allows you to:
- Tokenize (splitting strings in sub-word token strings), converttokens strings to ids and back, and encoding/decoding (i.e., tokenizing and converting to integers).
- Add new tokens to the vocabulary in a way that is independent of the underlying structure (BPE, SentencePiece…).
- Manage special tokens (like mask, beginning-of-sentence, etc.): adding them, assigning them to attributes in the tokenizer for easy access and making sure they are not split during tokenization.

Here we are going to use the tokenizer from the well known Bert model, that we can directly download.

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocessing_fn(x, tokenizer):
    x["review_ids"] = tokenizer(
        x["review"],
        add_special_tokens=False,
        truncation=True,
        max_length=256,
        padding=False,
        return_attention_mask=False,
    )["input_ids"]
    x["label"] = 0 if x["sentiment"] == "negative" else 1
    return x


Same celel than in the lab session.

🚧 **TODO** 🚧

Read the documentation about HuggingFace dataset and complete the code below.
You should:
- Shuffle the dataset
- For computational reasons, use only a total of **5000 samples**.
- Tokenize the dataset with the `preprocessing_fn`. (*Hint: use the `Dataset.map` method from HuggingFace*).
- Keep only columns `review_ids` and `label`.
- Make a train/validation split, (**80% / 20%**). Call these dataset `train_set` and `valid_set`.


In [ ]:
n_samples = 5000  # the number of training example

# We first shuffle the data !
dataset = dataset.shuffle()

# Select 5000 samples
dataset = dataset.select(range(n_samples))

# Tokenize the dataset
dataset = dataset.map(lambda x: preprocessing_fn(x, tokenizer))

# Remove useless columns
dataset = dataset.remove_columns(["review", "sentiment"])

# Split the train and validation
dataset = dataset.train_test_split(test_size=0.2)

document_train_set = dataset["train"]
document_valid_set = dataset["test"]


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [59]:
## QUESTION 2
def extract_words_context(id_list, R):
  ids = id_list[:]
  contexts = []

  for i in range(len(id_list)):
    # if we are at the border we add some padding, assuming 0 is the id of the token UNKNOWN
    if i < R:
      contexts += [[0]*(R-i) + id_list[:i] + id_list[i+1:R+i+1]]
    elif i>=len(id_list)-R:
      contexts += [id_list[i-R:i] + id_list[i+1:] + [0]*(R-len(id_list)+i+1)]
    else:
      contexts += [id_list[i-R:i] + id_list[i+1:i+R+1]]
  return ids, contexts

## QUESTION 3
def flatten_dataset_to_list(documentset,R):
  list_ids = []
  list_contexts = []
  for text in documentset["review_ids"]:
    new_ids, new_contexts = extract_words_context(text,R)
    list_ids += new_ids
    list_contexts += new_contexts
  return list_ids, list_contexts

## QUESTION 4
R = 10
flat_train = flatten_dataset_to_list(document_train_set, R)
flat_valid = flatten_dataset_to_list(document_valid_set, R)

print(len(flat_train))

2


In [ ]:
from torch.utils.data import Dataset

In [60]:
##QUESTION 5
class idDataset(Dataset):
  def __init__(self, pair_of_list):
    self.list_ids = pair_of_list[0]
    self.list_contexts = pair_of_list[1]

  def __len__(self):
    return len(self.list_ids)
  def __getitem__(self, idx):
    return self.list_ids[idx], self.list_contexts[idx]

train_set = idDataset(flat_train)
valid_set = idDataset(flat_valid)
print(train_set[2])


(3529, [0, 0, 0, 0, 0, 0, 0, 0, 1037, 20462, 2051, 3310, 2830, 2000, 2507, 1996, 3147, 2553, 3131, 2592])


In [63]:
## QUESTION 6
def collate_fn(batch : list[tuple[int, list[int]]], K : int, flat_dataset : tuple[list[int], list[list[int]]]):
  return dict(word_id = torch.tensor([id for id, _ in batch]),
              positive_context_ids= torch.tensor([cont for _, cont in batch]),
              negative_context_ids= torch.tensor([[flat_dataset[0][np.random.randint(len(flat_dataset))] for _ in range(len(batch[0][1])*K)]
                                                  for _ in range(len(batch))]))

## QUESTION 7
BATCH_SIZE = 10
K = 5
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=lambda batch: collate_fn(batch, K, flat_train))
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, collate_fn=lambda batch: collate_fn(batch, K, flat_valid))

## QUESTION 8
print("K : ", K)
print("R : ", R)
i = 0
for dic in train_loader:
  print("word_id size : ", dic["word_id"].size())
  print("positive_context_id size : ", dic["positive_context_ids"].size())
  print("negative_context_ids size : ", dic["negative_context_ids"].size())
  i += 1
  if i == 3:
    break


K :  5
R :  10
word_id size :  torch.Size([10])
positive_context_id size :  torch.Size([10, 20])
negative_context_ids size :  torch.Size([10, 100])
word_id size :  torch.Size([10])
positive_context_id size :  torch.Size([10, 20])
negative_context_ids size :  torch.Size([10, 100])
word_id size :  torch.Size([10])
positive_context_id size :  torch.Size([10, 20])
negative_context_ids size :  torch.Size([10, 100])


In [ ]:
## QUESTION 9
class Word2Vec(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
    self.context_embedding = nn.Embedding(vocab_size, embedding_dim)
    self.neg_context_embedding = nn.Embedding(vocab_size, embedding_dim)
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim

  def forward(self, word_id, positive_context_ids, negative_context_ids):
    word_embedding = self.word_embedding(word_id)
    positive_context_embedding = self.context_embedding(positive_context_ids)


762756
821284
